# Workflow , Module 

##

In [4]:
import requests
import pandas as pd

robokop_server = 'robokop.renci.org'

def quick(question):
    url=f'http://{robokop_server}:80/api/simple/quick/'
    response = requests.post(url,json=question)
    print( f"Return Status: {response.status_code}" )
    if response.status_code == 200:
        return response.json()
    return response

In [5]:
def make_N_step_question(types,curies):
    question = {
                'machine_question': {
                    'nodes': [],
                    'edges': []
                }
            }
    for i,t in enumerate(types):
        newnode = {'id': i, 'type': t}
        if curies[i] is not None:
            newnode['curie'] = curies[i]
        question['machine_question']['nodes'].append(newnode)
        if i > 0:
            question['machine_question']['edges'].append( {'source_id': i-1, 'target_id': i})
    return question

In [6]:
def extract_final_nodes(returnanswer):
    nodes = [{'node_name': answer['nodes'][2]['name'], 'node_id': answer['nodes'][2]['id']} for answer in returnanswer['answers']]
    return pd.DataFrame(nodes)

### Set up question

In [8]:
path=['disease','phenotype','genetic_condition']
starting_curie = 'MONDO:0004979' #asthma
curies = [starting_curie]
while len(curies) < len(path):
    curies.append(None)
question = make_N_step_question(path,curies)

In [15]:
answer = quick(question)

Return Status: 500


In [ ]:
frame = extract_final_nodes(answer)
frame